In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from pprint import pprint
import requests
import os
import re
import shutil # Utilites like copyfileobj()
import time
from time import sleep
import json

from pynput.mouse import Button, Controller as MouseCtrlr
from pynput.keyboard import Key, Controller as KeyCtrlr
import pyautogui as pag

from matplotlib import pyplot as plt
from PIL import Image
import pytesseract as tess
import cv2 as cv
import numpy as np
tess.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'

In [2]:
class BrowserWrapper:
    
    def __init__(self, driver, scriptDir = ""):
        self.driver = driver
        self.scriptDir = scriptDir
        
    
    def click(self, idStr=""):
        return self.driver.execute_script(f"""document.getElementById("{idStr}").click();""")
    
    def getHtml(self, idStr=""):
        return self.driver.execute_script(f"""document.getElementById("{idStr}").outerHTML;""")
    
    def run(self, code):
        return self.driver.execute_script(code)
    
    def runFile(self, path):
        code = ""
        with open(os.path.join(self.scriptDir, path + ".js")) as file:
            code = file.read()
            
        return self.driver.execute_script(code)
    
    def runFileJson(self, path):
        jsonData = self.runFile(path)
        parsed = None
        try:
            parsed = json.loads(jsonData)
        except Exception as e:
            print("Could not parse JSON: ", str(e))
        return parsed
    
    def get(self, url):
        self.driver.get(url)


# Get data for todays patients

In [4]:
driver = webdriver.Firefox()

In [5]:
bw = BrowserWrapper(driver, "js")

## Log in

In [6]:
bw.get(serverUrl)

In [13]:
def imshow(img, cmap=None, size=10):
    if type(img) == str:
        img = cv.imread(img)
    elif(not type(img) == np.ndarray):
        return
    
    fig = plt.figure(figsize=(size,size))
    ax = plt.Axes(fig, [0,0,1,1])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.imshow(img, cmap)

In [14]:
def get_inverted(image):
    return cv.bitwise_not(image)


# get grayscale image
def get_grayscale(image):
    return cv.cvtColor(image, cv.COLOR_BGR2GRAY)


#thresholding
def thresholding(image, threshold=180):
    return cv.threshold(image, threshold, 255, cv.THRESH_BINARY)[1]
    #return cv.adaptiveThreshold(image, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY,11,2)
    
def thresholdingAdaptive(image):
    return cv.adaptiveThreshold(image, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY,11,2)

# Interaction functions

In [15]:
kb = KeyCtrlr()

### Load images

In [16]:
class Pos:
    def __init__(self, x = 0, y = 0):
        self.x = x
        self.y = y
        
    def __str__(self):
        return f"({self.x}, {self.y})"
    
    def __repr__(self):
        return f"({self.x}, {self.y})"
    
    def __bool__(self):
        return True

In [17]:
def message(str):
    return pag.confirm(str)

In [18]:
globalWaitTime = 1
globalLimitTime = 30
globalWaitBeforeStart = 0.2

def getWaitAndLimit(beforeStart = -1, limit = -1, wait = -1):
    if wait == -1:
        wait = globalWaitTime
    if limit == -1:
        limit = globalLimitTime
    if beforeStart == -1:
        beforeStart = globalWaitBeforeStart
        
    return (beforeStart, limit, wait)

In [19]:
def imgPath(name):
    return os.path.join("Images", name + ".png")

imgNameList = ["SokValjPatient"]
imgDict = {}

for name in imgNameList:
    path = imgPath(name)
    imgDict[name] = Image.open(path)

imgDict["SokValjPatient"]

In [20]:
def waitForElement(name, beforeStart = -1, limit = -1, waitTime = -1, conf = 0.9, warn = True):
    timePassed = 0
    beforeStart, limit, waitTime = getWaitAndLimit(beforeStart, limit, waitTime)
    sleep(beforeStart)
        
    path = imgPath(name)
    #print(path)
    
    while timePassed < limit:
        try:
            x, y = pag.locateCenterOnScreen(path, confidence = conf)
            return Pos(x, y)
        except Exception as e:
            pass
            #print(str(e))
            
        sleep(waitTime)
        timePassed += waitTime
    
    if warn:
        message("Timed out waiting for element " + name)
    return False
        
#waitForElement("SokValjPatient")

In [21]:
def waitForElementDisappear(name, beforeStart = -1, limit = -1, waitTime = -1, conf = 0.9, warn = True):
    timePassed = 0
    beforeStart, limit, waitTime = getWaitAndLimit(beforeStart, limit, waitTime)
    sleep(beforeStart)
        
    path = imgPath(name)
    #print(path)
    
    while timePassed < limit:
        try:
            x, y = pag.locateCenterOnScreen(path, confidence = conf)
        except Exception as e:
            if (str(e) == "cannot unpack non-iterable NoneType object"):
                return True
            else:
                print(str(e))
            
        sleep(waitTime)
        timePassed += waitTime
    
    if warn:
        message("Timed out waiting for element disappear " + name)
    return False
        
waitForElementDisappear("notepad")

True

In [22]:
def waitForOneOf(nameList, beforeStart = -1, limit = -1, waitTime = -1, conf = 0.9, warn = True):
    timePassed = 0
    beforeStart, limit, waitTime = getWaitAndLimit(beforeStart, limit, waitTime)
    sleep(beforeStart)
    
    while timePassed < limit:
        for name in nameList:
            pos = waitForElement(name, beforeStart = 0, limit = 0.01, waitTime = 0.01, warn = False)
            timePassed += 0.01
            if pos:
                return pos
            
        sleep(waitTime)
        timePassed += waitTime
        
    if warn:
        message("Timed out waiting for element list " + str(nameList))
        
    return False
    
#waitForOneOf(["FleraIFoljd", "osrsakskod1IcdFalt", "sokDiagnos"], limit=2)

In [23]:
def waitAndClickOneOf(nameList, relX = 0, relY = 0, beforeStart = -1, limit = -1, waitTime = -1, conf = 0.9, warn = True, button = "left"):
    pos = waitForOneOf(nameList, beforeStart = beforeStart, limit = limit, waitTime = waitTime, conf = conf, warn = warn)
    
    if pos:
        pos.x += relX
        pos.y += relY
        
        if button == "left":
            pag.leftClick(pos.x, pos.y)
        elif button == "right":
            pag.rightClick(pos.x, pos.y)
            
        return pos
    
    return False


#waitAndClickOneOf(["FleraIFoljd", "osrsakskod1IcdFalt", "sokDiagnos"])

In [24]:
def waitAndClickElement(name, relX = 0, relY = 0, beforeStart = -1, limit = -1, waitTime = -1, conf = 0.9, warn = True, button="left"):
    pos = waitForElement(name, beforeStart, limit, waitTime, conf, warn=warn)
    
    if pos:
        pos.x += relX
        pos.y += relY
        #print(f'Moving to {pos.x}, {pos.y}')
        
        if button == "left":
            pag.leftClick(pos.x, pos.y)
        elif button == "right":
            pag.rightClick(pos.x, pos.y)
        return pos
    else:
        return False

#waitAndClickElement("SokValjPatient")

In [25]:
def waitAndClickXY(x, y, beforeStart = -1, waitTime = -1, button="left"):
    pos = Pos(x, y)
    return waitAndClick([pos], beforeStart = beforeStart, waitTime = waitTime, button=button)

def waitAndClick(positions, beforeStart = -1, waitTime = -1, button="left"):
    beforeStart, limit, waitTime = getWaitAndLimit(beforeStart = beforeStart, wait = waitTime)
    
    try:
        if type(positions) != list and type(positions) != tuple:
            positions = [positions]

        #print(f"Sleeping {beforeStart} before starting")
        sleep(beforeStart)
        for i, pos in enumerate(positions):
            if type(pos) == list or type(pos) == tuple:
                posObj = Pos(pos[0], pos[1])
            else:
                posObj = pos
            
            #print(text)
            if button == "left":
                pag.leftClick(posObj.x, posObj.y)
            elif button == "right":
                pag.rightClick(posObj.x, posObj.y)
            if i != len(positions) - 1:
                sleep(waitTime)
        
        return True
    except Exception as e:
        print("Faild at waitAndClick: " + str(e))
        return False
    

In [26]:
def waitAndType(texts, beforeStart = -1, waitTime = -1):
    beforeStart, limit, waitTime = getWaitAndLimit(beforeStart = beforeStart, wait = waitTime)
    
    try:
        if type(texts) == str:
            texts = [texts]

        sleep(beforeStart)
        for i, text in enumerate(texts):
            #print(text)
            kb.type(text)
            if i != len(texts) - 1:
                sleep(waitTime)
        
        return True
    except Exception as e:
        print("Faild at waitAndType: " + str(e))
        return False
        
        
#waitAndType(["hej", "hej2", "hej3"])
#waitAndType("hejdå")

In [27]:
def waitAndPress(key1, key2 = None, key3 = None, beforeStart = -1):
    beforeStart, limit, waitTime = getWaitAndLimit(beforeStart)
    
    sleep(beforeStart)
    
    try:
        if key2 and key3:
            pag.hotkey(key1, key2, key3)
        elif key2:
            pag.hotkey(key1, key2)
        else:
            pag.press(key1)
        return True
    except Exception as e:
        pag.confirm(f"Failed wait and press {str(key1)},{str(key2)},{str(key3)}")
        return False
    